# Deep Learning based Pipeline with Multichannel Inputs for Patent Classification

This notebook describes a deep learning pipeline for automatic patent classification with multichannel inputs.  A neural network model is trained with multichannel inputs namely embeddings of different segments of patent texts, and sparse linear
input of different metadata. <br> <br>
<img src="arch_0000.png" height="600" width="700">

<br>
In this notebook the classification task is a multi-class classification. The basic outline is:  <br>  <br>

- load patent dataset  <br>
- apply preprocessing tasks  <br>
- apply Tokenization process  <br>
- Load a pretraines word embeddings model  <br>
- prepare the embedding matrix for patent texts   <br>
- concatenated deep layers <br>
- train a deep neural network on the data  <br>
- show the results  <br>


#  Loading patent dataset


In [4]:
import pandas as pd

df = pd.read_csv("../datasets/allITPatTextWith_Metadata.csv",  encoding = "ISO-8859-1", error_bad_lines=False)
df.columns =['ID','TI','AB','TECHF','BACKG','SUMM','CLMS','ICM','AY','IPC','REF','PA','INV']

df.dropna(subset=['ICM'], inplace=True)


df.fillna(value='', inplace=True)

df.head()

,ID,TI,AB,TECHF,BACKG,SUMM,CLMS,ICM,AY,IPC,REF,PA,INV
0,EP2000017943-0,"Recognition code, particularly for a disk-shap...","A recognition code, particularly for a disk-li...",[1] The present invention relates to a recogni...,"[2] It is known that identification labels, ad...",[5] The aim of the present invention is to ove...,,G06K0019-06,2000,"[G06K0019-06, G06K0007-10]",,[Video System Italia S.r.l.],[Tassello Stefano]
1,EP2003016733-0,Optical pickup apparatus for recording/reprodu...,An optical pickup apparatus for reproducing in...,[1] The present invention relates to an optica...,"BACKGROUND OF THE INVENTION [2] Recently, as...",SUMMARY OF THE INVENTION[11] An object of the ...,['An optical pickup apparatus for recording an...,G11B0007-135,2000,"[G11B0007-135, G11B0007-125]",,[Konica Minolta Opto Inc.],"[Arai Norikazu, Kojima Toshiyuki, Kiriki To..."
2,EP2011009984-0,"Large capacity data sales mediation system, se...","An animation data sales mediation method, an a...",[1] The present invention relates to large cap...,BACKGROUND OF THE INVENTION Description of t...,SUMMARY OF THE INVENTION[20] The present inven...,['A large capacity data sales mediation system...,G07F0017-16,2001,"[G07F0017-16, G06Q0030-06, G06Q0020-10, G06Q00...","[JPHEI033290B, JPHEI08235759B, JPHEI10334048B,...",[NEC Corporation],[Maeda Koji]
3,PCT1997010546-0,BRIDGE FOR A CLIENT-SERVER ENVIRONMENT,A software bridge (300) is introduced between ...,1 BRIDGE FOR A CLIENT-SERVER ENVIRONMENT Fiel...,Background of the Invention Overview of Object...,45 Disclosure of the Invention Accordingly the...,['1. A bridge (300) for use between a client (...,G06F009-46,1996,"[G06F009-46, G06F009-44, G06F0009-44, G06F0009...",,"[INTERNATIONAL BUSINESS MACHINES CORPORATION, ...",[COLYER ADRIAN MARK]
4,PCT1998021641-0,PROCESSOR HAVING SECTIONS OPERATING AT DIFFERE...,A processor (250) including a first execution ...,PROCESSOR HAVING SECTIONS OPERATING AT DIFFERE...,Background of the Prior Art Fig. 1 illustrates...,SUMMARY OF THE INVENTION The invention provide...,,G06F001-32,1997,"[G06F001-32, G06F0001-08, G06F0009-30, G06F000...",,,"[SAGER DAVID J, FLETCHER THOMAS D, HINTON GLEN..."


# Applying preprocessing tasks on metadat of patent
Converting the metatadata such as inventors and assignees of each patent into a python list, then apply preprocessing task on each element in the list in order to remove undesired tokens.


In [2]:
%%time
#preprocess of list fields
#convert all IPCs in df into one list
def toList(s):
    """
    this method is to convert the list of IPCs in each row from a string to a python List
    """
    s  = s.translate ({ord(c): " " for c in "[]"})
    ss= []
    for cls in s.strip().split(','):
        ss.append(cls.strip())
    return ss

#apply toList method on all rows in the DF
df['PA'] = df['PA'].map(lambda pa :   toList(pa))
df['INV'] = df['INV'].map(lambda inv :   toList(inv))

df.head()  

CPU times: user 2.53 s, sys: 76.7 ms, total: 2.61 s
Wall time: 2.61 s


In [3]:
%%time

def metadataPreprocessing(input):
    newInput=' '
    for item in input:
        item = item.translate ({ord(c): " " for c in "!@#$%^&*()'[]{};:,./<>?\|`~°=\"+"})
        itms=' '
        for itm in item.split():
            itms= itms +' '+itm.strip()
        newInput = newInput + ' '+ itms.strip().replace(' ','_')
    return newInput.strip()

df['PA'] = df['PA'].map(lambda pa :   metadataPreprocessing(pa))
df['INV'] = df['INV'].map(lambda inv :   metadataPreprocessing(inv))

df.head()

CPU times: user 5.69 s, sys: 35.4 ms, total: 5.73 s
Wall time: 5.74 s


#  Applying preprocessing tasks on texts of patent
A simple preprocessing tasks such as tokenization, stopword removal, lemmatization, and converting letters into lower case are performed on each text section of each patent document.

In [4]:
#preprocessing 
standardStopwordFile = "sources/stopwords/stopwords-all.txt"

#loading terms from a file to a set
def get_terms_from_file(filePath):
    terms = set(line.strip() for line in open(filePath))
    return terms

#remove undiserd terms
def remove_terms(termSet, phrase):
    newPhrase = ""
    for term in phrase.split():
        if term.strip() not in termSet and len(term.strip())>2:
            newPhrase = newPhrase + " " + term.strip()



def clean_texts(doc):
    #Remove punctuation from texts
    doc = doc.translate ({ord(c): ' ' for c in "0123456789!@#$%^&*()'/[]{};:,./<>?\|`~°=\"+"})
    # split into tokens by white space
    tokens = doc.lower().strip().split()
    
    # filter out stop words
    stop_words = get_terms_from_file(standardStopwordFile)
    #generalStopwords = get_terms_from_file(generalWordsFile)

    
    tokens = [w.strip('-')  for w in tokens if  w not in stop_words ]
    # filter out short and long  tokens
    output = [word for word in tokens if len(word.strip()) > 2 and len(word) < 30 ]
    output = " ".join(output)
    #apply stemming
    #output = stem_text(output)
    return output



In [5]:
%%time
#apply simple preprocessing on text
df['TI'] = df['TI'].map(lambda line : clean_texts(line))
df['AB'] = df['AB'].map(lambda line : clean_texts(line))
df['TECHF'] = df['TECHF'].map(lambda line : clean_texts(line))
df['BACKG'] = df['BACKG'].map(lambda line : clean_texts(line))
df['SUMM'] = df['SUMM'].map(lambda line : clean_texts(line))
df['CLMS'] = df['CLMS'].map(lambda line : clean_texts(line))


df.head()

CPU times: user 271 µs, sys: 66 µs, total: 337 µs
Wall time: 327 µs


,ID,TI,AB,TECHF,BACKG,SUMM,CLMS,ICM,AY,IPC,REF,PA,INV
0,EP2000017943-0,recognition disk-shaped medium,recognition disk-like medium multimedia applic...,recognition disk-shaped medium multimedia appl...,identification labels adapted interpreted opti...,overcome drawbacks noted conventional types id...,,G06K0019-06,2000,"[G06K0019-06, G06K0007-10]",,Video_System_Italia_S_r_l,Tassello_Stefano
1,EP2003016733-0,optical pickup recording reproducing,optical pickup reproducing optical recording m...,optical pickup recording reproducing optical p...,recently practical short wavelength red laser ...,provide pickup recording reproducing optical r...,optical pickup recording reproducing optical m...,G11B0007-135,2000,"[G11B0007-135, G11B0007-125]",,Konica_Minolta_Opto_Inc,Arai_Norikazu_Kojima_Toshiyuki_Kiriki_Toshihik...
2,EP2011009984-0,large capacity sales mediation,animation sales mediation animation sales medi...,large capacity sales large capacity sales medi...,recent years distributing music network rapidl...,implemented consideration problems provide ani...,large capacity sales mediation terminal large ...,G07F0017-16,2001,"[G07F0017-16, G06Q0030-06, G06Q0020-10, G06Q00...","[JPHEI033290B, JPHEI08235759B, JPHEI10334048B,...",NEC_Corporation,Maeda_Koji
3,PCT1997010546-0,bridge client-server environment,software bridge introduced client client-serve...,bridge client-server environment distributed c...,overview object-oriented programming developme...,bridge client distributed object-oriented brid...,bridge client distributed object-oriented brid...,G06F009-46,1996,"[G06F009-46, G06F009-44, G06F0009-44, G06F0009...",,INTERNATIONAL_BUSINESS_MACHINES_CORPORATION_CO...,COLYER_ADRIAN_MARK
4,PCT1998021641-0,sections operating rates,core clocked perform operations clock frequenc...,sections operating rates high speed processors...,illustrates microprocessor microprocessor incl...,microprocessor levels sub-core clocked frequen...,,G06F001-32,1997,"[G06F001-32, G06F0001-08, G06F0009-30, G06F000...",,,SAGER_DAVID_J_FLETCHER_THOMAS_D_HINTON_GLENN_J...


#  Applying preprocessing tasks patent labels (main IPC codes)
The main IPC codes is considered to be the labels for the patent documents. we only consider the subclass level of the IPC code. 
Each label/class has at least more than 500 documents.

In [6]:
#process the ICM codes and #related-patents
df['ICM'] = df['ICM'].map(lambda icmCode : icmCode[:4])  

df_ICMs = df.groupby(['ICM'])
df_ICMs = df_ICMs.size().reset_index(name='Docs')

print(len(df_ICMs.ICM.unique()))
#filter out the rows with #docs less than N documents
df_ICMOut =  df_ICMs[df_ICMs['Docs'] >= 500]

#filter out rows of the original dataframe df accordding to df_ICMOut
ICMList = df_ICMOut['ICM'].tolist()
df = df[df.ICM.isin(ICMList)]

icmCount = df_ICMs.count().tolist()[0]

print( 'number of remaining documents in the dataset is: ',len(df))

print('Number of unique labels is: ', len(df.ICM.unique()))

581
number of remaining documents in the dataset is:  403726
Number of unique labels is:  42


# Randomly reorder a dataset by rows

In [7]:
from sklearn.utils import shuffle

df = shuffle(df)

df.head()

,ID,TI,AB,TECHF,BACKG,SUMM,CLMS,ICM,AY,IPC,REF,PA,INV
63956,PCT1997037347-0,closed loop servo focus,focus beam radiant energy detector beam output...,arrangements optical disc drives improved serv...,optical disc drives stored spiral concentric t...,primary extend operating range focus servo ope...,,G11B,1997,"[G11B007-09, G11B007-095, G11B0007-085, G11B00...",,,CESHKOVSKY_LUDWIG
351416,PCT2017010778-0,based preventive maintenance large,based preventive maintenance large based preve...,refers predicting aging large operating detect...,sensor-based power generation makes rapid main...,,collecting sensor sensors calculated slope fai...,G06Q,2016,"[G06Q0050-10, G06Q0010-00]",,,BAE_Suk_Joo
144383,PCT2017071542-0,,acquiring configured acquire stream photograph...,limited,artwith development mobile phone mobile phone ...,outlined outlined non-is scope protection impr...,conversion configured chirp current frame carr...,H04N,2016,"[H04N0005-243, G06T0005-50, H04N0005-217, H04N...",,,ZHU_Dezhi
358317,PCT2012012538-0,memory cell,systems methods forming memory cells memory ce...,,memory cell cross-references applications cfr ...,aspects ultra-thin sram cells layout topologie...,,G06F,2011,[G06F0019-00],,UNIVERSITY_OF_VIRGINIA_PATENT_FOUNDATION_CALHO...,CALHOUN_Benton_H_MANN_Randy_W
363786,PCT2012015702-0,methods systems articles manufacture implement...,methods systems articles manufacture implement...,,methods systems articles manufacture implement...,presents methods systems products implementing...,implemented implementing electronic design ele...,G06F,2011,"[G06F0015-04, G06F0017-50]",,CADENCE_DESIGN_SYSTEMS_INC_GOPALAKRISHNAN_Prak...,GOPALAKRISHNAN_Prakash_MCSHERRY_Michael_WHITE_...


# split the dataset into train and test datasets

In [8]:
# lets take n% data as training and remaining m% for test.
train_size = int(len(df) * .8)

train_TI = df['TI'][:train_size]
train_AB = df['AB'][:train_size]
train_TECHF = df['TECHF'][:train_size]
train_BACKG = df['BACKG'][:train_size]
train_SUMM = df['SUMM'][:train_size]
train_CLMS = df['CLMS'][:train_size]
train_ICM= df['ICM'][:train_size]
train_ID= df['ID'][:train_size]

test_TI = df['TI'][train_size:]
test_AB = df['AB'][train_size:]
test_TECHF = df['TECHF'][train_size:]
test_BACKG = df['BACKG'][train_size:]
test_SUMM = df['SUMM'][train_size:]
test_CLMS = df['CLMS'][train_size:]
test_ICM = df['ICM'][train_size:]
test_ID = df['ID'][train_size:]


#metadata
train_pa_series = df['PA'][:train_size]
test_pa_series = df['PA'][train_size:]

train_inv_series = df['INV'][:train_size]
test_inv_series = df['INV'][train_size:]


print(train_AB.shape)
print(test_AB.shape)

#free up some memory space
#df.iloc[0:0]

(363353,)
(40373,)


#  Applying tokenization process 
For texts of each segment, a Keras tokenization process is used for breaking the text into individual words, and  set the sequence length of each segment according to the length of each.


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelBinarizer


Using TensorFlow backend.


Applying Keras tokenization on Metadata of patent(Inventors, Assignees), and convert the related text into One-hot that encodes a text into a list of word indexes of size n.

In [10]:
#PA
pa_inv_vocab_size = 2000
pa_tokenizer = Tokenizer(num_words=pa_inv_vocab_size,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~', lower=True, split=' ', char_level=False, oov_token=None)
pa_tokenizer.fit_on_texts(train_pa_series)
train_pa_one_hot =pa_tokenizer.texts_to_matrix(train_pa_series)
test_pa_one_hot =pa_tokenizer.texts_to_matrix(test_pa_series)


#INV
inv_tokenizer = Tokenizer(num_words=pa_inv_vocab_size,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~', lower=True, split=' ', char_level=False, oov_token=None)
inv_tokenizer.fit_on_texts(train_inv_series)
train_inv_one_hot =inv_tokenizer.texts_to_matrix(train_inv_series)
test_inv_one_hot =inv_tokenizer.texts_to_matrix(test_inv_series)


print('Found %s words in PA' % len(pa_tokenizer.word_index))
print('Found %s words in INV' % len(inv_tokenizer.word_index))

Found 97734 words in PA
Found 280346 words in INV


<br>
Transform each text in Titles (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [11]:
%%time

#Title
TI_tokenizer = Tokenizer(num_words=10000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
TI_tokenizer.fit_on_texts(train_TI)
encoded_train_TI = TI_tokenizer.texts_to_sequences(train_TI)
encoded_test_TI = TI_tokenizer.texts_to_sequences(test_TI)
#convert all sequences in a list into the same length
TI_train = pad_sequences(encoded_train_TI,  maxlen=20, padding='post')
TI_test = pad_sequences(encoded_test_TI,  maxlen=20, padding='post')


CPU times: user 11 s, sys: 69.9 ms, total: 11.1 s
Wall time: 11.1 s


<br>
Transform each text in Abstrcat (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [12]:
%%time
#Abstract
AB_tokenizer = Tokenizer(num_words=50000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
AB_tokenizer.fit_on_texts(train_AB)
encoded_train_AB = AB_tokenizer.texts_to_sequences(train_AB)
encoded_test_AB = AB_tokenizer.texts_to_sequences(test_AB)
#convert all sequences in a list into the same length
AB_train = pad_sequences(encoded_train_AB,  maxlen=100, padding='post')
AB_test = pad_sequences(encoded_test_AB,  maxlen=100, padding='post')


CPU times: user 30.3 s, sys: 377 ms, total: 30.7 s
Wall time: 30.6 s


<br>
Transform each text in Technical Field (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [13]:
%%time
#TECHNICAL_FIELD
TECHF_tokenizer = Tokenizer(num_words=20000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
TECHF_tokenizer.fit_on_texts(train_TECHF)
encoded_train_TECHF = TECHF_tokenizer.texts_to_sequences(train_TECHF)
encoded_test_TECHF = TECHF_tokenizer.texts_to_sequences(test_TECHF)
#convert all sequences in a list into the same length
TECHF_train = pad_sequences(encoded_train_TECHF,  maxlen=30, padding='post')
TECHF_test = pad_sequences(encoded_test_TECHF,  maxlen=30, padding='post')


CPU times: user 19 s, sys: 148 ms, total: 19.2 s
Wall time: 19.2 s


<br>
Transform each text in Background (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [14]:
%%time
#BACKGROUND
BACKG_tokenizer = Tokenizer(num_words=50000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
BACKG_tokenizer.fit_on_texts(train_BACKG)
encoded_train_BACKG = BACKG_tokenizer.texts_to_sequences(train_BACKG)
encoded_test_BACKG = BACKG_tokenizer.texts_to_sequences(test_BACKG)
#convert all sequences in a list into the same length
BACKG_train = pad_sequences(encoded_train_BACKG,  maxlen=100, padding='post')
BACKG_test = pad_sequences(encoded_test_BACKG,  maxlen=100, padding='post')


CPU times: user 2min 6s, sys: 1.12 s, total: 2min 7s
Wall time: 2min 7s


<br>
Transform each text in Summary (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [15]:
%%time
#SUMMARY
SUMM_tokenizer = Tokenizer(num_words=50000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
SUMM_tokenizer.fit_on_texts(train_SUMM)
encoded_train_SUMM = SUMM_tokenizer.texts_to_sequences(train_SUMM)
encoded_test_SUMM = SUMM_tokenizer.texts_to_sequences(test_SUMM)
#convert all sequences in a list into the same length
SUMM_train = pad_sequences(encoded_train_SUMM,  maxlen=100, padding='post')
SUMM_test = pad_sequences(encoded_test_SUMM,  maxlen=100, padding='post')


CPU times: user 2min 46s, sys: 1.53 s, total: 2min 47s
Wall time: 2min 47s


<br>
Transform each text in Independent Claim (train and test datasets) into a sequence of integers. <br>
set the sequence length.<br>
Pads sequences to the same length.<br>



In [16]:
%%time
#CLAIMS
CLMS_tokenizer = Tokenizer(num_words=50000,  filters='!"#$%&()*+,./:;<=>?@[\]^`{|}~_', lower=True, split=' ', char_level=False, oov_token=None)
CLMS_tokenizer.fit_on_texts(train_CLMS)
encoded_train_CLMS = CLMS_tokenizer.texts_to_sequences(train_CLMS)
encoded_test_CLMS = CLMS_tokenizer.texts_to_sequences(test_CLMS)
#convert all sequences in a list into the same length
CLMS_train = pad_sequences(encoded_train_CLMS,  maxlen=100, padding='post')
CLMS_test = pad_sequences(encoded_test_CLMS,  maxlen=100, padding='post')


CPU times: user 40.3 s, sys: 792 ms, total: 41 s
Wall time: 41.1 s


<br>
Representing the labels/classes in the numeric format by scikit-learn - LabelBinarizer class. <br>
Convert 1-dimensional class arrays to n-dimensional(#classes) class matrices.

In [17]:
%%time
# 
encoder = LabelBinarizer()
encoder.fit(train_ICM)
y_train = encoder.transform(train_ICM)
y_test = encoder.transform(test_ICM)

#get the unique number of labels in the training set
classesList = train_ICM.tolist()
classesList =set(classesList)
num_classes = len(classesList)

CPU times: user 4.04 s, sys: 64.8 ms, total: 4.1 s
Wall time: 4.11 s


#  load the whole embeddings model into memory and get matrix
We load a pre-trained word2vec word embedding model that was trained on five million patents (Titles and abstracts)

In [18]:
import numpy as np

def load_embedding_model(filePath):
    embeddings_index = dict()
    f = open(filePath, encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
    return embeddings_index

def create_embedding_matrix(tokenizer, embeddings_index, vocab_size_embbs, dim_size):
    embeddings_matrix = np.zeros((vocab_size_embbs, dim_size))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector[0:dim_size]
    
    return embeddings_matrix


 Loading the whole embedding into memory and get matrix

In [19]:
%%time

embedding_index = load_embedding_model('../models/w2v/phrase/patWordPhrase2VecModel.txt')


CPU times: user 1min 31s, sys: 2.75 s, total: 1min 34s
Wall time: 1min 34s


<br>
Creating TITLE embedding Matrix

In [20]:
%%time

#vocab_size for embedding
vocab_size_embb = len(TI_tokenizer.word_index) + 1

TI_embeddings_matrix = create_embedding_matrix(TI_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              20)




CPU times: user 86.3 ms, sys: 57.9 ms, total: 144 ms
Wall time: 143 ms



Creating ABSTRACT embedding Matrix

In [21]:
%%time

#vocab_size for embedding
vocab_size_embb = len(AB_tokenizer.word_index) + 1
AB_embeddings_matrix = create_embedding_matrix(AB_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              100)


CPU times: user 235 ms, sys: 116 ms, total: 351 ms
Wall time: 351 ms


Creating TECHNICAL_FIELD embedding Matrix

In [22]:
%%time

#vocab_size for embedding
vocab_size_embb = len(TECHF_tokenizer.word_index) + 1
TECHF_embeddings_matrix = create_embedding_matrix(TECHF_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              30)

CPU times: user 161 ms, sys: 26 ms, total: 187 ms
Wall time: 187 ms


Creating BACKGROUND embedding Matrix

In [ ]:
%%time

#vocab_size for embedding
vocab_size_embb = len(BACKG_tokenizer.word_index) + 1
BACKG_embeddings_matrix = create_embedding_matrix(BACKG_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              100)

Creating SUMMARY embeddings Matrix

In [24]:
%%time

#vocab_size for embedding
vocab_size_embb = len(SUMM_tokenizer.word_index) + 1
SUMM_embeddings_matrix = create_embedding_matrix(SUMM_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              100)

CPU times: user 540 ms, sys: 155 ms, total: 695 ms
Wall time: 696 ms


Creating INDEPENDENT CLAIMS embeddings Matrix

In [25]:
%%time

#vocab_size for embedding
vocab_size_embb = len(CLMS_tokenizer.word_index) + 1
CLMS_embeddings_matrix = create_embedding_matrix(CLMS_tokenizer,
                                              embedding_index,
                                              vocab_size_embb,
                                              100)

CPU times: user 187 ms, sys: 46.9 ms, total: 234 ms
Wall time: 234 ms


# Creating a Deep Layer for each Input

In [26]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Embedding, BatchNormalization, ELU, Concatenate
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers.core import Dropout


<br> Creating LSTM deep layer for Title Embeddings<br>

In [27]:
%%time
#TITLE 
sequence_len =20
dropout_pct =  0.3

TI_embedding_layer_input = Input(shape=(sequence_len,), name='TI_embed_input')
TI_embedding_layer = Embedding(input_dim=len(TI_tokenizer.word_index) + 1,
                        output_dim=20, # Dimension of the dense embedding
                        weights=[TI_embeddings_matrix],
                        input_length=20)(TI_embedding_layer_input)

lstm_size = 64
TI_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_TI')(TI_embedding_layer)

TI_deep = Dense(300, activation=None)(TI_deep)
TI_deep = Dropout(dropout_pct)(TI_deep)
TI_deep = BatchNormalization()(TI_deep)
TI_deep = ELU()(TI_deep)


CPU times: user 4.52 s, sys: 8.61 s, total: 13.1 s
Wall time: 4.08 s


<br> Creating LSTM deep layer for Abstract Embeddings<br>


In [28]:
%%time
#Abstract 
sequence_len =100
dropout_pct =  0.3

AB_embedding_layer_input = Input(shape=(sequence_len,), name='AB_embed_input')
AB_embedding_layer = Embedding(input_dim=len(AB_tokenizer.word_index) + 1,
                        output_dim=100, # Dimension of the dense embedding
                        weights=[AB_embeddings_matrix],
                        input_length=100)(AB_embedding_layer_input)

lstm_size = 64
AB_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_AB')(AB_embedding_layer)

AB_deep = Dense(300, activation=None)(AB_deep)
AB_deep = Dropout(dropout_pct)(AB_deep)
AB_deep = BatchNormalization()(AB_deep)
AB_deep = ELU()(AB_deep)


CPU times: user 3.46 s, sys: 6.4 s, total: 9.86 s
Wall time: 791 ms


<br> Creating LSTM deep layer for TECHNICAL-Field Embeddings<br>


In [29]:
%%time
#TECHNICAL-Field 
sequence_len =30
dropout_pct =  0.3

TECHF_embedding_layer_input = Input(shape=(sequence_len,), name='TECHF_embed_input')
TECHF_embedding_layer = Embedding(input_dim=len(TECHF_tokenizer.word_index) + 1,
                        output_dim=30, # Dimension of the dense embedding
                        weights=[TECHF_embeddings_matrix],
                        input_length=30)(TECHF_embedding_layer_input)

lstm_size = 64
TECHF_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_TECHF')(TECHF_embedding_layer)

TECHF_deep = Dense(300, activation=None)(TECHF_deep)
TECHF_deep = Dropout(dropout_pct)(TECHF_deep)
TECHF_deep = BatchNormalization()(TECHF_deep)
TECHF_deep = ELU()(TECHF_deep)

CPU times: user 3.54 s, sys: 6.37 s, total: 9.9 s
Wall time: 744 ms


<br> Creating LSTM deep layer for BACKGROUND Embeddings<br>

In [30]:
%%time
#BACKGROUND 
sequence_len =100
dropout_pct =  0.3

BACKG_embedding_layer_input = Input(shape=(sequence_len,), name='BACKG_embed_input')
BACKG_embedding_layer = Embedding(input_dim=len(BACKG_tokenizer.word_index) + 1,
                        output_dim=100, # Dimension of the dense embedding
                        weights=[BACKG_embeddings_matrix],
                        input_length=100)(BACKG_embedding_layer_input)

lstm_size = 64
BACKG_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_BACK')(BACKG_embedding_layer)

BACKG_deep = Dense(300, activation=None)(BACKG_deep)
BACKG_deep = Dropout(dropout_pct)(BACKG_deep)
BACKG_deep = BatchNormalization()(BACKG_deep)
BACKG_deep = ELU()(BACKG_deep)

CPU times: user 6.76 s, sys: 7.53 s, total: 14.3 s
Wall time: 5.29 s


<br> Creating LSTM deep layer for Summary Embeddings<br>

In [31]:
%%time
#SUMMARY
sequence_len =100
dropout_pct =  0.3

SUMM_embedding_layer_input = Input(shape=(sequence_len,), name='SUMM_embed_input')
SUMM_embedding_layer = Embedding(input_dim=len(SUMM_tokenizer.word_index) + 1,
                        output_dim=100, # Dimension of the dense embedding
                        weights=[SUMM_embeddings_matrix],
                        input_length=100)(SUMM_embedding_layer_input)

lstm_size = 64
SUMM_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_SUMM')(SUMM_embedding_layer)

SUMM_deep = Dense(300, activation=None)(SUMM_deep)
SUMM_deep = Dropout(dropout_pct)(SUMM_deep)
SUMM_deep = BatchNormalization()(SUMM_deep)
SUMM_deep = ELU()(SUMM_deep)

CPU times: user 3.74 s, sys: 6.75 s, total: 10.5 s
Wall time: 1.3 s


<br> Creating LSTM deep layer for Independent Claim Embeddings<br>

In [32]:
%%time
#CLAIMS 
sequence_len =100
dropout_pct =  0.4


CLMS_embedding_layer_input = Input(shape=(sequence_len,), name='CLMS_embed_input')
CLMS_embedding_layer = Embedding(input_dim=len(CLMS_tokenizer.word_index) + 1,
                        output_dim=100, # Dimension of the dense embedding
                        weights=[CLMS_embeddings_matrix],
                        input_length=100)(CLMS_embedding_layer_input)

lstm_size = 64
CLMS_deep = LSTM(lstm_size,
            dropout=dropout_pct,
            recurrent_dropout=dropout_pct,
            return_sequences=False,
            name='LSTM_CLMS')(CLMS_embedding_layer)

CLMS_deep = Dense(300, activation=None)(CLMS_deep)
CLMS_deep = Dropout(dropout_pct)(CLMS_deep)
CLMS_deep = BatchNormalization()(CLMS_deep)
CLMS_deep = ELU()(CLMS_deep)

CPU times: user 3.66 s, sys: 6.42 s, total: 10.1 s
Wall time: 1.02 s


<br> Creating LSTM deep layers for one-hot vectors of Inventors and Assignees<br>

In [33]:
dropout_pct =  0.3
pa_input = Input(shape=(train_pa_one_hot.shape[1],), name='pa_input') 
pas = Dense(32,input_dim=train_pa_one_hot.shape[1], activation=None)(pa_input) 
pas = Dropout(dropout_pct)(pas)
pas = BatchNormalization()(pas)
pas = ELU()(pas)

#inv
inv_input = Input(shape=(train_inv_one_hot.shape[1],), name='inv_input') 
invs = Dense(32,input_dim=train_inv_one_hot.shape[1], activation=None)(pa_input) 
invs = Dropout(dropout_pct)(invs)
invs = BatchNormalization()(invs)

print('pa_input and inv_input layers are finished')

pa_input and inv_input layers are finished


# Train Model
The following cells specify the neural network architecture and hyperparameters.

The model is generally composed of:

contacting sequential word embeddings ofpatent text segments into a fully-connected layer
Compile the  the Network

In [34]:
import keras_metrics as km
#contacting 
model_inputs_to_concat = [TI_deep, AB_deep, TECHF_deep, BACKG_deep, SUMM_deep, CLMS_deep] #invs , pas, invs
final_layer =  Concatenate(name='concatenated_layer')(model_inputs_to_concat)

output = Dense(128, activation=None)(final_layer)
output = Dropout(dropout_pct)(output)
output = BatchNormalization()(output)
output = ELU()(output)
output = Dense(num_classes, activation='sigmoid')(output)

model = Model(inputs=[TI_embedding_layer_input,
                      AB_embedding_layer_input,
                      TECHF_embedding_layer_input,
                      BACKG_embedding_layer_input,
                     SUMM_embedding_layer_input,
                     CLMS_embedding_layer_input,
                     ],
              outputs=output, name='model')
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy', km.categorical_precision(), km.categorical_recall()])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TI_embed_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
AB_embed_input (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
TECHF_embed_input (InputLayer)  (None, 30)           0                                            
__________________________________________________________________________________________________
BACKG_embed_input (InputLayer)  (None, 100)          0                                            
__________________________________________________________________________________________________
SUMM_embed

Train / Fit the Network

In [35]:
%%time
batch_size= 1000 
num_epochs = 20

history = model.fit(x={'TI_embed_input': TI_train,
                       'AB_embed_input': AB_train,
             'TECHF_embed_input': TECHF_train,
             'BACKG_embed_input': BACKG_train,
             'SUMM_embed_input': SUMM_train,
             'CLMS_embed_input': CLMS_train
             
            },
          y=y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=
          ({'TI_embed_input': TI_test,
            'AB_embed_input': AB_test,
            'TECHF_embed_input': TECHF_test,
             'BACKG_embed_input': BACKG_test,
             'SUMM_embed_input': SUMM_test,
            'CLMS_embed_input': CLMS_test
            },
           y_test))


Train on 363353 samples, validate on 40373 samples
Epoch 1/20
363353/363353 [==============================] - 234s 645us/step - loss: 2.1174 - acc: 0.4668 - precision: 0.0544 - recall: 0.4532 - val_loss: 1.4309 - val_acc: 0.5763 - val_precision: 0.2264 - val_recall: 0.5767
Epoch 2/20
363353/363353 [==============================] - 225s 620us/step - loss: 1.4617 - acc: 0.5754 - precision: 0.2742 - recall: 0.4366 - val_loss: 1.3037 - val_acc: 0.6040 - val_precision: 0.2569 - val_recall: 0.7522
Epoch 3/20
363353/363353 [==============================] - 221s 609us/step - loss: 1.3580 - acc: 0.5978 - precision: 0.3420 - recall: 0.4350 - val_loss: 1.2414 - val_acc: 0.6108 - val_precision: 0.2933 - val_recall: 0.7566
Epoch 4/20
363353/363353 [==============================] - 222s 610us/step - loss: 1.3011 - acc: 0.6083 - precision: 0.3827 - recall: 0.4097 - val_loss: 1.2194 - val_acc: 0.6165 - val_precision: 0.3459 - val_recall: 0.7035
Epoch 5/20
363353/363353 [===========================

In [36]:
from sklearn.datasets import make_circles
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

<br>Train the neural network on  multichannel inputs namely deep layers of patent text segments 
 and deep layers of patent metadata.<br>

In [40]:
import keras_metrics as km

#contacting two input models
model_inputs_to_concat = [TI_deep, AB_deep, TECHF_deep, BACKG_deep, SUMM_deep, CLMS_deep, pas, invs] 
final_layer =  Concatenate(name='concatenated_layer')(model_inputs_to_concat)

output = Dense(128, activation=None)(final_layer)
output = Dropout(dropout_pct)(output)
output = BatchNormalization()(output)
output = ELU()(output)
output = Dense(num_classes, activation='sigmoid')(output)

model2 =Model(inputs=[ TI_embedding_layer_input,
                      AB_embedding_layer_input,
                      TECHF_embedding_layer_input,
                      BACKG_embedding_layer_input,
                     SUMM_embedding_layer_input,
                     CLMS_embedding_layer_input,
                     pa_input,
                      inv_input],
              outputs=output, name='model')
model2.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                       metrics=['accuracy', km.categorical_precision(), km.categorical_recall()])
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TI_embed_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
AB_embed_input (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
TECHF_embed_input (InputLayer)  (None, 30)           0                                            
__________________________________________________________________________________________________
BACKG_embed_input (InputLayer)  (None, 100)          0                                            
__________________________________________________________________________________________________
SUMM_embed

Fit the model/network

In [41]:
%%time
batch_size= 1000 
num_epochs = 10


history2 = model2.fit(x={'TI_embed_input': TI_train,
                         'AB_embed_input': AB_train,
             'TECHF_embed_input': TECHF_train,
             'BACKG_embed_input': BACKG_train,
             'SUMM_embed_input': SUMM_train,
             'CLMS_embed_input': CLMS_train,
             'pa_input': train_pa_one_hot,
             'inv_input': train_inv_one_hot
            },
          y=y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=
          ({'TI_embed_input': TI_test,
            'AB_embed_input': AB_test,
            'TECHF_embed_input': TECHF_test,
             'BACKG_embed_input': BACKG_test,
             'SUMM_embed_input': SUMM_test,
            'CLMS_embed_input': CLMS_test,
            'pa_input': test_pa_one_hot,
            'inv_input': test_inv_one_hot
            },
           y_test))

Train on 363353 samples, validate on 40373 samples
Epoch 1/10
363353/363353 [==============================] - 306s 842us/step - loss: 1.5099 - acc: 0.6580 - precision: 0.0953 - recall: 0.7308 - val_loss: 1.0988 - val_acc: 0.6680 - val_precision: 0.4294 - val_recall: 0.7448
Epoch 2/10
363353/363353 [==============================] - 289s 795us/step - loss: 0.9267 - acc: 0.7100 - precision: 0.5445 - recall: 0.5696 - val_loss: 1.0764 - val_acc: 0.6687 - val_precision: 0.4055 - val_recall: 0.7876
Epoch 3/10
363353/363353 [==============================] - 292s 805us/step - loss: 0.8939 - acc: 0.7165 - precision: 0.5801 - recall: 0.5254 - val_loss: 1.0814 - val_acc: 0.6704 - val_precision: 0.4621 - val_recall: 0.6932
Epoch 4/10
363353/363353 [==============================] - 290s 797us/step - loss: 0.8745 - acc: 0.7199 - precision: 0.5860 - recall: 0.5117 - val_loss: 1.0844 - val_acc: 0.6672 - val_precision: 0.4615 - val_recall: 0.7080
Epoch 5/10
363353/363353 [===========================

<br>Train the neural network on  multichannel inputs namely deep layers of Title and Technical Field.<br>

In [42]:


#contacting two input models
model_inputs_to_concat = [TI_deep,  TECHF_deep] #invs , pas, invs
final_layer =  Concatenate(name='concatenated_layer')(model_inputs_to_concat)

output = Dense(64, activation=None)(final_layer)
output = Dropout(dropout_pct)(output)
output = BatchNormalization()(output)
output = ELU()(output)
output = Dense(num_classes, activation='sigmoid')(output)

model3 =Model(inputs=[ TI_embedding_layer_input,
                      TECHF_embedding_layer_input
                      ],
              outputs=output, name='model')
model3.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                       metrics=['accuracy', km.categorical_precision(), km.categorical_recall()])
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TI_embed_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
TECHF_embed_input (InputLayer)  (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 20)       914260      TI_embed_input[0][0]             
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 30, 30)       4662210     TECHF_embed_input[0][0]          
__________________________________________________________________________________________________
LSTM_TI (L

<br> Fit the model <br>

In [43]:
%%time
batch_size= 1000 
num_epochs = 10


history3 = model3.fit(x={'TI_embed_input': TI_train,
             'TECHF_embed_input': TECHF_train
            },
          y=y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=
          ({'TI_embed_input': TI_test,
            'TECHF_embed_input': TECHF_test
            },
           y_test))

Train on 363353 samples, validate on 40373 samples
Epoch 1/10
363353/363353 [==============================] - 31s 86us/step - loss: 2.0806 - acc: 0.5398 - precision: 0.0570 - recall: 0.6855 - val_loss: 1.4180 - val_acc: 0.5989 - val_precision: 0.2578 - val_recall: 0.6077
Epoch 2/10
363353/363353 [==============================] - 26s 71us/step - loss: 1.4311 - acc: 0.5970 - precision: 0.3425 - recall: 0.4566 - val_loss: 1.3698 - val_acc: 0.6009 - val_precision: 0.2964 - val_recall: 0.6313
Epoch 3/10
363353/363353 [==============================] - 25s 70us/step - loss: 1.3923 - acc: 0.6019 - precision: 0.4039 - recall: 0.4379 - val_loss: 1.3576 - val_acc: 0.5997 - val_precision: 0.3446 - val_recall: 0.5885
Epoch 4/10
363353/363353 [==============================] - 26s 70us/step - loss: 1.3739 - acc: 0.6036 - precision: 0.4367 - recall: 0.3838 - val_loss: 1.3461 - val_acc: 0.6010 - val_precision: 0.3758 - val_recall: 0.5310
Epoch 5/10
363353/363353 [==============================] - 2

<br>Train the neural network on  multichannel inputs namely deep layers of Title, abstrcat and Technical Field.<br>

In [44]:
import keras_metrics as km
#contacting two input models
model_inputs_to_concat = [TI_deep, AB_deep, TECHF_deep] #invs , pas, invs
final_layer =  Concatenate(name='concatenated_layer')(model_inputs_to_concat)

output = Dense(128, activation=None)(final_layer)
output = Dropout(dropout_pct)(output)
output = BatchNormalization()(output)
output = ELU()(output)
output = Dense(num_classes, activation='sigmoid')(output)

model4 = Model(inputs=[TI_embedding_layer_input,
                      AB_embedding_layer_input,
                      TECHF_embedding_layer_input
                     ],
              outputs=output, name='model')
model4.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                       metrics=['accuracy', km.categorical_precision(), km.categorical_recall()])
model4.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TI_embed_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
AB_embed_input (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
TECHF_embed_input (InputLayer)  (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 20)       914260      TI_embed_input[0][0]             
__________________________________________________________________________________________________
embedding_

<br>Fit the model <br>

In [45]:
%%time
batch_size= 1000 
num_epochs = 10


history4 = model4.fit(x={'TI_embed_input': TI_train,
                         'AB_embed_input': AB_train,
             'TECHF_embed_input': TECHF_train
            },
          y=y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=
          ({'TI_embed_input': TI_test,
            'AB_embed_input': AB_test,
            'TECHF_embed_input': TECHF_test
            },
           y_test))

Train on 363353 samples, validate on 40373 samples
Epoch 1/10
363353/363353 [==============================] - 85s 233us/step - loss: 1.7639 - acc: 0.5874 - precision: 0.0978 - recall: 0.6676 - val_loss: 1.2810 - val_acc: 0.6221 - val_precision: 0.3321 - val_recall: 0.6386
Epoch 2/10
363353/363353 [==============================] - 77s 212us/step - loss: 1.2674 - acc: 0.6251 - precision: 0.4109 - recall: 0.5129 - val_loss: 1.2499 - val_acc: 0.6228 - val_precision: 0.3627 - val_recall: 0.6195
Epoch 3/10
363353/363353 [==============================] - 77s 211us/step - loss: 1.2374 - acc: 0.6298 - precision: 0.4551 - recall: 0.4477 - val_loss: 1.2355 - val_acc: 0.6254 - val_precision: 0.3602 - val_recall: 0.6268
Epoch 4/10
363353/363353 [==============================] - 78s 213us/step - loss: 1.2186 - acc: 0.6332 - precision: 0.4769 - recall: 0.4159 - val_loss: 1.2249 - val_acc: 0.6228 - val_precision: 0.3630 - val_recall: 0.6195
Epoch 5/10
363353/363353 [==============================]

<br>Train the neural network on  multichannel inputs namely deep layers of Title, Technical Field, Inventors, and Assignees.<br>

In [46]:


#contacting two input models
model_inputs_to_concat = [TI_deep, TECHF_deep,  pas, invs] #invs , pas, invs
final_layer =  Concatenate(name='concatenated_layer')(model_inputs_to_concat)

output = Dense(128, activation=None)(final_layer)
output = Dropout(dropout_pct)(output)
output = BatchNormalization()(output)
output = ELU()(output)
output = Dense(num_classes, activation='sigmoid')(output)

model5 =Model(inputs=[ TI_embedding_layer_input,
                      TECHF_embedding_layer_input,
                     pa_input,
                      inv_input],
              outputs=output, name='model')
model5.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                       metrics=['accuracy', km.categorical_precision(), km.categorical_recall()])
model5.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TI_embed_input (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
TECHF_embed_input (InputLayer)  (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 20)       914260      TI_embed_input[0][0]             
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 30, 30)       4662210     TECHF_embed_input[0][0]          
__________________________________________________________________________________________________
LSTM_TI (L

<br< Fit the mode. <br>

In [47]:
%%time
batch_size= 1000 
num_epochs = 10


history5 = model5.fit(x={'TI_embed_input': TI_train,
             'TECHF_embed_input': TECHF_train,
             'pa_input': train_pa_one_hot,
             'inv_input': train_inv_one_hot
            },
          y=y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=
          ({'TI_embed_input': TI_test,
            'TECHF_embed_input': TECHF_test,
            'pa_input': test_pa_one_hot,
            'inv_input': test_inv_one_hot
            },
           y_test))

Train on 363353 samples, validate on 40373 samples
Epoch 1/10
363353/363353 [==============================] - 38s 106us/step - loss: 1.7714 - acc: 0.5898 - precision: 0.0911 - recall: 0.6943 - val_loss: 1.2946 - val_acc: 0.6199 - val_precision: 0.3291 - val_recall: 0.6445
Epoch 2/10
363353/363353 [==============================] - 33s 90us/step - loss: 1.2540 - acc: 0.6284 - precision: 0.4267 - recall: 0.5499 - val_loss: 1.2791 - val_acc: 0.6155 - val_precision: 0.3236 - val_recall: 0.6534
Epoch 3/10
363353/363353 [==============================] - 33s 90us/step - loss: 1.2313 - acc: 0.6310 - precision: 0.4637 - recall: 0.4969 - val_loss: 1.2716 - val_acc: 0.6154 - val_precision: 0.3373 - val_recall: 0.6209
Epoch 4/10
363353/363353 [==============================] - 32s 89us/step - loss: 1.2190 - acc: 0.6328 - precision: 0.4732 - recall: 0.4597 - val_loss: 1.2731 - val_acc: 0.6143 - val_precision: 0.3538 - val_recall: 0.6047
Epoch 5/10
363353/363353 [==============================] - 